# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import string 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# Load Dataset Teks 

Dataset yang digunakan hanyalah tweet yang berupa konten narasi. 

In [ ]:
df = pd.read_csv('data_search.csv', usecols=['Tweet'])
df

,Tweet
0,"TERJANGKAU, Hubungi 0812-1244-2489, Kavling Ta..."
1,akwowakoawkaow kos gawe di jakarta wae
2,"Damage nya ga parah banget nih nder , take all..."
3,RT : Goodbener mengajak kita semua warga DKI J...
4,10# Berikut delapan dukungan yang disampaikan ...
...,...
95,SAKING OTAK NYA GA MAMPU DAN GA PUNYA GAGASAN ...
96,CARI TANAH KAVLING UNTUK BANGUN KEBUN BUAH? TE...
97,"RT : . 19 March 2023 // Jakarta, Indonesia Tic..."
98,"4# Kenneth mengatakan, pemerintah pusat selalu..."


# Data Preprocessing

## Case Folding

Tahap case folding merupakan tahapan mengubah semua karakter yang terdapat dalam dokumen teks menjadi huruf kecil. 

In [ ]:
df['Tweet'] = df['Tweet'].str.lower()
df.head(20)

,Tweet
0,"terjangkau, hubungi 0812-1244-2489, kavling ta..."
1,akwowakoawkaow kos gawe di jakarta wae
2,"damage nya ga parah banget nih nder , take all..."
3,rt : goodbener mengajak kita semua warga dki j...
4,10# berikut delapan dukungan yang disampaikan ...
5,rt : pejabat jika sudah tercap pembohong pleh ...
6,rt : salah satu lukisan yang dipamerkan di pam...
7,berikut jadwal imsakiyah 1443 h untuk wilayah ...
8,sudahkah kamu join grup hari ini? dm gan kalo ...
9,_ dokgi jakarta di tempat praktek yg tepat jem...


## Tokenizing

Tahap tokenizing merupakan tahap yang dilakukan setelah teks melewati tahap case folding. Pada tahap tokenizing teks melalui proses pemecahan perkata dan proses menghilangkan angka, tanda baca, whitespace, dan satu karakter.

In [ ]:
def remove_tweet_special(text):
    # menghapus tab, garis baru, dan backslash
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # menghapus non ASCII (emotikon, kata-kata china, dll)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['Tweet'] = df['Tweet'].apply(remove_tweet_special)

#menghapus angka
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Tweet'] = df['Tweet'].apply(remove_number)

#menghapus tanda baca
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['Tweet'] = df['Tweet'].apply(remove_punctuation)

#menghapus whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Tweet'] = df['Tweet'].apply(remove_whitespace_LT)

#menghapus whitespace ganda menjadi satu whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Tweet'] = df['Tweet'].apply(remove_whitespace_multiple)

# menghapus satu karakter
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Tweet'] = df['Tweet'].apply(remove_single_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenization'] = df['Tweet'].apply(word_tokenize_wrapper)
df['tokenization'].head(20)

0     [terjangkau, hubungi, kavling, tanah, dijual, ...
1         [akwowakoawkaow, kos, gawe, di, jakarta, wae]
2     [damage, nya, ga, parah, banget, nih, nder, ta...
3     [rt, goodbener, mengajak, kita, semua, warga, ...
4     [berikut, delapan, dukungan, yang, disampaikan...
5     [rt, pejabat, jika, sudah, tercap, pembohong, ...
6     [rt, salah, satu, lukisan, yang, dipamerkan, d...
7     [berikut, jadwal, imsakiyah, untuk, wilayah, d...
8     [sudahkah, kamu, join, grup, hari, ini, dm, ga...
9     [dokgi, jakarta, di, tempat, praktek, yg, tepa...
10                  [jakarta, untung, udah, bali, wkwk]
11    [rt, org, orang, jakarta, mungkin, tidak, tahu...
12    [terlaris, hubungi, jual, kavling, tanah, mura...
13    [rt, idha, festival, ramadhan, dan, telur, pas...
14       [jungwon, lenti, down, aja, kak, dom, jakarta]
15    [rt, base, jakarta, todays, cawang, jakarta, t...
16    [demi, terciptanya, herd, immunity, polres, ja...
17                        [ppkm, jakarta, dimana

## Filtering (Stopword Removal)

Tahap filtering (Stopword Removal) merupakan tahapan setelah teks melalui proses pemecahan perkata pada tahap tokenizing. Tahapan filtering bertujuan untuk mengambil kata-kata yang dianggap penting untuk proses selanjutnya. Kata-kata umum yang biasanya muncul dalam jumlah besar dan tidak memiliki makna disebut stopword. Kata-kata stopword seperti “yang”, “dan”, “di”, “dari”, dan sejenisnya perlu dihilangkan dalam teks sehingga menyisakan kata-kata yang dianggap penting.

In [ ]:
list_stopwords = stopwords.words("indonesian")
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", "klo", 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['filter_tokens'] = df['tokenization'].apply(stopwords_removal) 
df['filter_tokens'].head(20)

0     [terjangkau, hubungi, kavling, tanah, dijual, ...
1             [akwowakoawkaow, kos, gawe, jakarta, wae]
2     [damage, parah, banget, nder, take, all, poton...
3     [goodbener, mengajak, warga, dki, jakarta, men...
4     [delapan, dukungan, anies, dibutuhkan, jakarta...
5     [pejabat, tercap, pembohong, pleh, rakyat, lak...
6     [salah, lukisan, dipamerkan, pameran, internas...
7     [jadwal, imsakiyah, wilayah, dki, jakarta, sel...
8                                 [join, grup, dm, gan]
9     [dokgi, jakarta, praktek, jem, kerja, part, ti...
10                  [jakarta, untung, udah, bali, wkwk]
11    [org, orang, jakarta, kota, mengalami, kehilan...
12    [terlaris, hubungi, jual, kavling, tanah, mura...
13    [idha, festival, ramadhan, telur, paskah, bers...
14            [jungwon, lenti, down, kak, dom, jakarta]
15    [base, jakarta, todays, cawang, jakarta, timut...
16    [terciptanya, herd, immunity, polres, jakarta,...
17                              [ppkm, jakarta, 

## Normalisasi Kata

Tahap normalisasi kata merupakan tahapan setelah teks melalui proses pemecahan perkata pada tahap tokenizing. Tahapan normalisasi bertujuan untuk menyeragamkan kata-kata yang memiliki makna sama namun penulisannya berbeda. Kata-kata yang perlu dinormalisasikan merupakan kata-kata yang bisa diakibatkan kesalahan penulisan, penyingkatan kata, atau penggunaan bahasa gaul seperti “kenapa” yang ditulis “knp” atau “knapa”, dan sejenisnya perlu diseragamkan menjadi satu kata yang memiliki makna yang sama. 

In [ ]:
normalizad_word = pd.read_csv("normalisasi.csv")
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalized'] = df['filter_tokens'].apply(normalized_term)
df['normalized'].head(20)

0     [terjangkau, hubungi, kavling, tanah, dijual, ...
1             [akwowakoawkaow, kos, gawe, jakarta, wae]
2     [damage, parah, banget, nder, take, all, poton...
3     [goodbener, mengajak, warga, dki, jakarta, men...
4     [delapan, dukungan, anies, dibutuhkan, jakarta...
5     [pejabat, tercap, pembohong, pleh, rakyat, lak...
6     [salah, lukisan, dipamerkan, pameran, internas...
7     [jadwal, imsakiyah, wilayah, dki, jakarta, sel...
8                                 [join, grup, dm, gan]
9     [dokgi, jakarta, praktek, jem, kerja, part, ti...
10                  [jakarta, untung, udah, bali, wkwk]
11    [orang, orang, jakarta, kota, mengalami, kehil...
12    [terlaris, hubungi, jual, kavling, tanah, mura...
13    [idha, festival, ramadhan, telur, paskah, bers...
14            [jungwon, lenti, down, kak, dom, jakarta]
15    [base, jakarta, todays, cawang, jakarta, timut...
16    [terciptanya, herd, immunity, polres, jakarta,...
17                              [ppkm, jakarta, 

## Stemming

Tahap stemming merupakan tahapan setelah teks melalui proses filtering dan normalisasi kata. Tahapan stemming bertujuan untuk menghilangkan imbuhan depan dan imbuhan belakang pada teks sehingga teks kembali kebentuk dasarnya. Kata-kata yang perlu dilakukan tahap stemming diantaranya “pertolongan” menjadi “tolong”, “kebersihan” menjadi “bersih”, dan sejenisnya. 

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['stemmed'] = df['normalized'].swifter.apply(get_stemmed_term)
df['stemmed'].head(20)

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

0     [jangkau, hubung, kavling, tanah, jual, murah,...
1             [akwowakoawkaow, kos, gawe, jakarta, wae]
2     [damage, parah, banget, nder, take, all, poton...
3     [goodbener, ajak, warga, dki, jakarta, gapai, ...
4     [delapan, dukung, anies, butuh, jakarta, perin...
5        [jabat, cap, bohong, pleh, rakyat, laku, malu]
6     [salah, lukis, pamer, pamer, internasional, ka...
7     [jadwal, imsakiyah, wilayah, dki, jakarta, sel...
8                                 [join, grup, dm, gan]
9     [dokgi, jakarta, praktek, jem, kerja, part, ti...
10                   [jakarta, untung, udah, bal, wkwk]
11    [orang, orang, jakarta, kota, alami, hilang, j...
12    [laris, hubung, jual, kavling, tanah, murah, k...
13    [idha, festival, ramadhan, telur, paskah, sand...
14            [jungwon, lenti, down, kak, dom, jakarta]
15    [base, jakarta, todays, cawang, jakarta, timut...
16    [cipta, herd, immunity, polres, jakarta, utara...
17                                [ppkm, jakarta

# Saving Pandas Dataframe to CSV File

In [41]:
df.to_csv('data_prep.csv')